# Import packages

In [234]:
import numpy as np
import pandas as pd
import keras
import tensorflow
import matplotlib.pyplot as plt

from keras import backend as K
from keras import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Flatten, Conv1D
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix, classification_report
from keras import optimizers

# Import data

In [12]:
data = pd.read_csv('data/creditcard.csv')

In [13]:
data.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [14]:
data.Class.value_counts()

0    284315
1       492
Name: Class, dtype: int64

# Handle imbalance dataset by proportion of nonfraud to fraud(1=1:1,2=2:1,3=3:1,...)

In [150]:
def undersampling_data_prep(df, prop): 
    fraud = df[df.Class == 1]
    nonfraud = df[df.Class == 0]
    nonfraud = nonfraud.sample(len(fraud) * prop)
    new_data = fraud.append(nonfraud)
    
    X = new_data.drop('Class', axis = 1)
    y = new_data.Class

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

    scalar = StandardScaler()
    X_train=scalar.fit_transform(X_train)
    X_test=scalar.transform(X_test)
    return X_train, X_test, y_train, y_test

In [166]:
def oversampling_data_prep(df, prop): 
    fraud = df[df.Class == 1]
    nonfraud = df[df.Class == 0]
    times = int(len(nonfraud) / len(fraud) * (1 / prop))
    for i in range(times):
        nonfraud = nonfraud.append(fraud)
    
    X = nonfraud.drop('Class', axis = 1)
    y = nonfraud.Class

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

    scalar = StandardScaler()
    X_train=scalar.fit_transform(X_train)
    X_test=scalar.transform(X_test)
    return X_train, X_test, y_train, y_test

# Custom Report Function

In [220]:
def report(model, trainX, testX, trainY, testY, prop):
    clf = model
    clf.fit(trainX, trainY)
    predY = clf.predict(testX)
    m = confusion_matrix(testY, predY)
    print("------------------------------------")
    print("Fraudulent transaction proportion: " + str(1 / (1 + prop)))
    print("Non-Fraudulent transaction proportion: " + str(1 - 1 / (1 + prop)))
    print("------------------------------------")
    TP = m[1,1]
    TN = m[0,0]
    FP = m[0,1]
    FN = m[1,0]
    Precision = TP / (TP + FP)
    Recall = TP / (TP + FN)
    F1 = 2 * Precision * Recall / (Precision + Recall)
    Acc = (TP + TN) / (TP + TN + FP + FN)
    avg = (Recall + Precision + F1 + Acc) / 4
    print("Classification report: ")
    print("Recall: %.2f" % Recall)
    print("Precision: %.2f " % Precision)
    print("F1 score: %.2f" % F1)
    print("Accuracy: %.2f" % Acc)
    print("Average score: %.2f" % avg)
    print()
    return [Recall, Precision, F1, Acc, avg]

# Random Forest Classifier

In [225]:
RF = RandomForestClassifier()
RF_score = []
for i in range(1,10):
    X_train, X_test, y_train, y_test = undersampling_data_prep(data,i)
    RF_score.append(report(RF, X_train, X_test, y_train, y_test, i))

------------------------------------
Fraudulent transaction proportion: 0.5
Non-Fraudulent transaction proportion: 0.5
------------------------------------
Classification report: 
Recall: 0.92
Precision: 0.99 
F1 score: 0.95
Accuracy: 0.95
Average score: 0.95

------------------------------------
Fraudulent transaction proportion: 0.3333333333333333
Non-Fraudulent transaction proportion: 0.6666666666666667
------------------------------------
Classification report: 
Recall: 0.92
Precision: 1.00 
F1 score: 0.96
Accuracy: 0.97
Average score: 0.96

------------------------------------
Fraudulent transaction proportion: 0.25
Non-Fraudulent transaction proportion: 0.75
------------------------------------
Classification report: 
Recall: 0.89
Precision: 0.97 
F1 score: 0.93
Accuracy: 0.96
Average score: 0.94

------------------------------------
Fraudulent transaction proportion: 0.2
Non-Fraudulent transaction proportion: 0.8
------------------------------------
Classification report: 
Recal

# Logistic Regression Classifier

In [226]:
LR = LogisticRegression()
LR_score = []
for i in range(1,10):
    X_train, X_test, y_train, y_test = data_prep(data,i)
    LR_score.append(report(LR, X_train, X_test, y_train, y_test, i))

------------------------------------
Fraudulent transaction proportion: 0.5
Non-Fraudulent transaction proportion: 0.5
------------------------------------
Classification report: 
Recall: 0.91
Precision: 0.97 
F1 score: 0.94
Accuracy: 0.94
Average score: 0.94

------------------------------------
Fraudulent transaction proportion: 0.3333333333333333
Non-Fraudulent transaction proportion: 0.6666666666666667
------------------------------------
Classification report: 
Recall: 0.92
Precision: 0.97 
F1 score: 0.94
Accuracy: 0.96
Average score: 0.95

------------------------------------
Fraudulent transaction proportion: 0.25
Non-Fraudulent transaction proportion: 0.75
------------------------------------
Classification report: 
Recall: 0.89
Precision: 0.99 
F1 score: 0.94
Accuracy: 0.97
Average score: 0.95

------------------------------------
Fraudulent transaction proportion: 0.2
Non-Fraudulent transaction proportion: 0.8
------------------------------------
Classification report: 
Recal

# Decision Tree Classifier

In [228]:
DT = DecisionTreeClassifier()
DT_score = []
for i in range(1,10):
    X_train, X_test, y_train, y_test = data_prep(data,i)
    DT_score.append(report(DT, X_train, X_test, y_train, y_test, i))

------------------------------------
Fraudulent transaction proportion: 0.5
Non-Fraudulent transaction proportion: 0.5
------------------------------------
Classification report: 
Recall: 0.93
Precision: 0.91 
F1 score: 0.92
Accuracy: 0.92
Average score: 0.92

------------------------------------
Fraudulent transaction proportion: 0.3333333333333333
Non-Fraudulent transaction proportion: 0.6666666666666667
------------------------------------
Classification report: 
Recall: 0.92
Precision: 0.92 
F1 score: 0.92
Accuracy: 0.95
Average score: 0.93

------------------------------------
Fraudulent transaction proportion: 0.25
Non-Fraudulent transaction proportion: 0.75
------------------------------------
Classification report: 
Recall: 0.88
Precision: 0.88 
F1 score: 0.88
Accuracy: 0.94
Average score: 0.89

------------------------------------
Fraudulent transaction proportion: 0.2
Non-Fraudulent transaction proportion: 0.8
------------------------------------
Classification report: 
Recal

# ANN

In [218]:
def recall_m(y_true, y_pred):
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    PP = K.sum(K.round(K.clip(y_true, 0, 1)))
    return TP / (PP + K.epsilon())

def precision_m(y_true, y_pred):
    TP = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    PP = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return TP / (PP + K.epsilon())

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision * recall)/(precision + recall + K.epsilon()))

def NN(Xt):
    model = keras.Sequential()
    model.add(Dense(128, activation='relu', input_shape=(Xt.shape[-1],)))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(128, activation='relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy', recall_m, precision_m, f1_m])
    
    return model

def NN_report(model):
    print("------------------------------------")
    print("Fraudulent transaction proportion: " + str(1 / (1 + i)))
    print("Non-Fraudulent transaction proportion: " + str(1 - 1 / (1 + i)))
    print("------------------------------------")
    history = model.fit(X_train, y_train, batch_size = 64, epochs = 200, verbose = 0)
    score = model.evaluate(X_test, y_test)
    avg = (score[1] + score[2] + score[3] + score[4]) / 4
    print("Classification report: ")
    print("Recall: %.2f" % score[2])
    print("Precision: %.2f " % score[3])
    print("F1 score: %.2f" % score[4])
    print("Accuracy: %.2f" % score[1]) 
    print("Average score: %.2f" % avg)
    print()
    return [score[2], score[3], score[4], score[1], avg]

In [229]:
NN_score = []
for i in range(1,10):
    X_train, X_test, y_train, y_test = data_prep(data,i)
    model = NN(Xt = X_train)
    NN_score.append(NN_report(model))

------------------------------------
Fraudulent transaction proportion: 0.5
Non-Fraudulent transaction proportion: 0.5
------------------------------------
7/7 [==============================] - 0s 1ms/step - loss: 0.2668 - accuracy: 0.9391 - recall_m: 0.9412 - precision_m: 0.9528 - f1_m: 0.9457
Classification report: 
Recall: 0.94
Precision: 0.95 
F1 score: 0.95
Accuracy: 0.94
Average score: 0.94

------------------------------------
Fraudulent transaction proportion: 0.3333333333333333
Non-Fraudulent transaction proportion: 0.6666666666666667
------------------------------------
10/10 [==============================] - 1s 988us/step - loss: 0.1748 - accuracy: 0.9595 - recall_m: 0.9261 - precision_m: 0.9537 - f1_m: 0.9368
Classification report: 
Recall: 0.93
Precision: 0.95 
F1 score: 0.94
Accuracy: 0.96
Average score: 0.94

------------------------------------
Fraudulent transaction proportion: 0.25
Non-Fraudulent transaction proportion: 0.75
------------------------------------
13/1

# CNN

In [237]:
def CNN(Xt):
    model=Sequential()
    model.add(Conv1D(128,2,activation = 'relu',input_shape=Xt[0].shape))
    model.add(BatchNormalization())
    model.add(Dropout(0.2))

    model.add(Conv1D(128,2,activation = 'relu'))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Flatten())
    model.add(Dense(128,activation = 'relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1,activation = 'sigmoid'))
    
    model.compile(optimizer = 'adam', loss='binary_crossentropy', metrics = ['accuracy', recall_m, precision_m, f1_m])
    
    return model

def CNN_report(model):
    print("------------------------------------")
    print("Fraudulent transaction proportion: " + str(1 / (1 + i)))
    print("Non-Fraudulent transaction proportion: " + str(1 - 1 / (1 + i)))
    print("------------------------------------")
    history = model.fit(X_train, y_train, batch_size = 64, epochs = 200, verbose = 0)
    score = model.evaluate(X_test, y_test)
    avg = (score[1] + score[2] + score[3] + score[4]) / 4
    print("Classification report: ")
    print("Recall: %.2f" % score[2])
    print("Precision: %.2f " % score[3])
    print("F1 score: %.2f" % score[4])
    print("Accuracy: %.2f" % score[1]) 
    print("Average score: %.2f" % avg)
    print()
    return [score[2], score[3], score[4], score[1], avg]

In [238]:
CNN_score = []
for i in range(1,10):
    X_train, X_test, y_train, y_test = data_prep(data,i)
    y_train = y_train.to_numpy()
    y_test = y_test.to_numpy()
    X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
    X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)
    model = CNN(Xt = X_train)
    CNN_score.append(CNN_report(model))   

------------------------------------
Fraudulent transaction proportion: 0.5
Non-Fraudulent transaction proportion: 0.5
------------------------------------
7/7 [==============================] - 1s 3ms/step - loss: 0.4297 - accuracy: 0.9442 - recall_m: 0.9412 - precision_m: 0.9695 - f1_m: 0.9539
Classification report: 
Recall: 0.94
Precision: 0.97 
F1 score: 0.95
Accuracy: 0.94
Average score: 0.95

------------------------------------
Fraudulent transaction proportion: 0.3333333333333333
Non-Fraudulent transaction proportion: 0.6666666666666667
------------------------------------
10/10 [==============================] - 1s 3ms/step - loss: 0.4655 - accuracy: 0.9527 - recall_m: 0.9184 - precision_m: 0.9367 - f1_m: 0.9239
Classification report: 
Recall: 0.92
Precision: 0.94 
F1 score: 0.92
Accuracy: 0.95
Average score: 0.93

------------------------------------
Fraudulent transaction proportion: 0.25
Non-Fraudulent transaction proportion: 0.75
------------------------------------
13/13 